In [98]:
import numpy as numpy
import random as random


num_queen=8
population_num=100
mutation_prob=0.2
max_fitness=28 ##sum of all the row positions of queens when no diagonal or horizontal attacks [0,1,2,3,4,5,6,7] 


class ChessBoard:
    def __init__(self):
        self.chromosome = None
        self.fitness_score = None
        self.prob_score = None
    def setChromosomeVal(self,val):
        self.chromosome=val
    def setFitnessScoreVal(self,val):
        self.fitness_score=val
    def setProbScoreVal(self,val):
        self.prob_score=val
    def getAttr(self):
        return {'chromosome':chromosome, 'fitness_score':fitness_score, 'prob_score':prob_score}
        

def random_gen(num_queen):
    chromosome=[random.randint(0,num_queen-1) for i in range(num_queen)]
    #print(chromosome)
    return chromosome


def gen_population():

    population = [ChessBoard() for i in range(population_num)]
    for i in range(population_num):
        chromosome=random_gen(num_queen)
        population[i].setChromosomeVal(chromosome)
        
        h_attack=horizontal_attack(chromosome)
        diag_attack=diagonal_attack(chromosome)
        fit_score=cal_fitness_score(h_attack,diag_attack)
        population[i].setFitnessScoreVal(fit_score)
    return population


def horizontal_attack(chrom_list):
    h_attack=(len(chrom_list)-len(list(set(chrom_list))))
    return h_attack

def diagonal_attack(chromosome):
    n = len(chromosome)
    left_diagonal = [0] * 2*n
    right_diagonal = [0] * 2*n
    for i in range(n):
        left_diagonal[i + chromosome[i] - 1] += 1
        right_diagonal[len(chromosome) - i + chromosome[i] - 2] += 1

    diagonal_collisions = 0
    for i in range(2*n-1):
        counter = 0
        if left_diagonal[i] > 1:
            counter += left_diagonal[i]-1
        if right_diagonal[i] > 1:
            counter += right_diagonal[i]-1
        diagonal_collisions += counter / (n-abs(i-n+1))
        diag_attack=diagonal_collisions
    
    #for i in range(len(chrom_list)):
     #   diag_attack=0
      #  
       # for j in range(len(chrom_list)):
        #    if (i != j):
         #       x_dist=abs(chrom_list[i]-chrom_list[j])
          #      y_dist=abs(i-j)
           #     if (x_dist==y_dist):
            #        diag_attack=diag_attack+1
                    

   
    return diag_attack
    

def cal_fitness_score(horizontal_attack ,diag_attack):
    total_attack=horizontal_attack+diag_attack
    fit_score=max_fitness - pow(total_attack,2)
    return fit_score

def cal_fit_total_population(population):
    total_fitness=numpy.sum([pop.fitness_score for pop in population])
    
    return total_fitness

def cal_probability(population,Total_Fitness_Score):
    relative_fitness = [population[i].fitness_score/Total_Fitness_Score for i in range(len(population))]
    probabilities = [sum(relative_fitness[:i+1]) for i in range(len(relative_fitness))]
    for i in range(len(probabilities)):
                   population[i].setProbScoreVal(probabilities[i])
        
        #print(probability)
    

def parent_selection(population,Total_Fitness_score):
    parent_chrom= None
    
    r_value= numpy.random.rand()
    #print(r_value)
    parent=[pop for pop in population if pop.prob_score >= r_value]
    #print(len(parent))
    #print(parent[0])
    while True:
        try:
            parent_chrom=parent[0]
            break
        except:
            pass
        #print(r_value)
    #print(parent_chrom.prob_score)
    return parent_chrom
    
    
def child_crossover(parent1,parent2):
    length=len(parent1.chromosome)
    c=random.randint(0,length-1)
    
    return parent1.chromosome[0:c] + parent2.chromosome[c:length]
    
def mutate_fn(chrom):
    length= len(chrom)
    cut_pos = random.randint(0, length -1 )
    mutat_val = random.randint(1, length-1)
    chrom[cut_pos] = mutat_val
    return chrom

def gen_child(population,Total_Fitness_Score):
    print("gen child started")
    new_population=[]
    child =  ChessBoard() 
    for i in range(population_num):
       
       # print(Total_Fitness_Score)
       # print ("Execution for ", i )
        parent_1=parent_selection(population,Total_Fitness_Score)
        #print("Parent 1 is ", parent_1)
        parent_2=parent_selection(population,Total_Fitness_Score)
        #print("Parent 2 is ", parent_2)
        child_chrom=child_crossover(parent_1,parent_2)
        if random.random() < mutation_prob:
            child_chrom = mutate_fn(child_chrom)
        #child=ChessBoard()
       # print(child_chrom)
        child.setChromosomeVal(child_chrom)
        child_h_attack=horizontal_attack(child_chrom)
        child_diag_attack =diagonal_attack(child_chrom)
        child_fit_score=cal_fitness_score(child_h_attack,child_diag_attack)
        child.setFitnessScoreVal(child_fit_score)
        new_population.append(child)
        if child_fit_score == max_fitness: break
    return new_population


if __name__ == "__main__":

    population = gen_population()
    generation = 1
    Total_Fitness_Score=cal_fit_total_population(population)
    print(Total_Fitness_Score)
    print("===  Total_Fitness_Score {} ===".format(Total_Fitness_Score))
    cal_probability(population,Total_Fitness_Score)
    
    while not max_fitness in [pop.fitness_score for pop in population]:
        print("===  Generation {} ===".format(generation))
        print("")
        print("")
        print(Total_Fitness_Score)
        print ("Start chil_crossover")
        print("")
        new_population = gen_child(population, Total_Fitness_Score)
        print ("length of child population is " , len(new_population))
        #new_population.getAttr()
        #print(new_population)
        total_score=0;
        #for pop in new_population:
         #   print ( "Fitness score for ", pop.chromosome, " is ", pop.fitness_score)
          #  total_score+=pop.fitness_score
        Total_Fitness_Score=numpy.sum([pop.fitness_score for pop in new_population]) #cal_fit_total_population(new_population)
        cal_probability(new_population,Total_Fitness_Score)
        max_new_fitness=numpy.max([n.fitness_score for n in new_population])
        print("Maximum Fitness = {}".format(max([n.fitness_score for n in new_population])))
        if max_new_fitness == max_fitness:
            break
        generation += 1
    chrom_out = []
    print("Solved in Generation {}!".format(generation-1))
    for chrom in new_population:
        if cal_fitness_score(horizontal_attack(chrom.chromosome),diagonal_attack(chrom.chromosome)) == max_fitness:
            print("");
            print("Solution for n-queens problem: ")
            chrom_out = chrom
            #print(chrom.chromosome)
            listToStr = ' '.join(map(str, chrom.chromosome)) 
            print(listToStr)  
            break
            
            
   

1643.028173185941
===  Total_Fitness_Score 1643.028173185941 ===
===  Generation 1 ===


1643.028173185941
Start chil_crossover

gen child started
length of child population is  100
Maximum Fitness = 21.059284297052155
===  Generation 2 ===


2105.928429705216
Start chil_crossover

gen child started
length of child population is  100
Maximum Fitness = 6.133242630385485
===  Generation 3 ===


613.3242630385482
Start chil_crossover

gen child started
length of child population is  100
Maximum Fitness = 21.059284297052155
===  Generation 4 ===


2105.928429705216
Start chil_crossover

gen child started
length of child population is  100
Maximum Fitness = 8.334998582766442
===  Generation 5 ===


833.4998582766442
Start chil_crossover

gen child started
length of child population is  100
Maximum Fitness = 21.90968112244898
===  Generation 6 ===


2190.968112244898
Start chil_crossover

gen child started
length of child population is  100
Maximum Fitness = 12.378684807256235
===  Generatio